In [ ]:
#r "D:\BoSSS2\experimental\public\src\L4-application\BoSSSpad\bin\Release\net5.0\bossspad.dll"
// #r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using System.IO;
using System.Data;
using System.Globalization;
using System.Threading;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Platform.LinAlg;
using BoSSS.Solution.NSECommon;
using BoSSS.Application.XNSEC;
Init();

In [ ]:
BoSSSshell.WorkflowMgm.Init("XDG_Droplet_Combustionn");
static var myBatch = GetDefaultQueue();
static var myDb = myBatch.CreateOrOpenCompatibleDatabase("XDG_Droplet_Combustionn");
BoSSSshell.WorkflowMgm.DefaultDatabase = myDb;

Project name is set to 'XDG_Droplet_Combustionn'.
Creating database '\\hpccluster\hpccluster-scratch\gutierrez\XDG_Droplet_Combustionn'.


In [ ]:
BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation()

## Case configuration

In [ ]:
int nCells = 5; // Mesh Resolution
int dgMF = 2; //DG degree
static bool m_BotPressureOutlet = true; // swtiches for changing boundary conditions
static bool m_TopPressureOutlet = false;

## Create grid

In [ ]:
public static class GridFactory {

    public static Grid2D GenerateGrid1D(int nCells) {
        double LLL = 30; // Length of the domain
        double h = Math.Pow(2, -nCells + 1); // cell length
        double cells = 1 / h;
        int cells2 = (int)cells;
        
        var _xNodes = GenericBlas.Linspace(0, 5,  4 ); // 4 cells at least are needed 
        var _yNodes = GenericBlas.Linspace(0, LLL, cells2 + 1);
        var grd = Grid2D.Cartesian2DGrid(_xNodes, _yNodes, periodicX: true);

        if (m_BotPressureOutlet) {
            grd.EdgeTagNames.Add(1, "ScalarDirichlet_PressureOutlet_bot");
        } else {
            grd.EdgeTagNames.Add(1, "velocity_inlet_bot");
        }

        if (m_TopPressureOutlet) {
            grd.EdgeTagNames.Add(2, "ScalarDirichlet_PressureOutlet_top");
        } else {
            grd.EdgeTagNames.Add(2, "velocity_inlet_top");
        }

        
        grd.DefineEdgeTags(delegate (double[] X) {
            double x = X[0];
            double y = X[1];
            //upper Wall
            if (Math.Abs(y - LLL) < 1e-8)
                return 2;

            //lower Wall
            if (Math.Abs(y + 0) < 1e-8)
                return 1;
            else throw new ArgumentOutOfRangeException();
        });
        myDb.SaveGrid(ref grd);

        return grd;
    }
 


}

In [ ]:
public static class BoundaryValueFactory { 

    public static string GetPrefixCode(double ConstVal) {
        using(var stw = new System.IO.StringWriter()) {

            stw.WriteLine("static class BoundaryValues {");

            stw.WriteLine("  static public double ConstantValue(double[] X) {");
            stw.WriteLine("    return "+ ConstVal +";");
            stw.WriteLine("  }");

            stw.WriteLine("  static public double Phi(double[] X) {");
            stw.WriteLine("    return X[1] - "+ ConstVal +";"); 
            stw.WriteLine("  }");


            stw.WriteLine("}"); 
            return stw.ToString();
        }
    }

    static public Formula Get_ConstantValue(double ConstVal){
        return new Formula("BoundaryValues.ConstantValue", AdditionalPrefixCode:GetPrefixCode(ConstVal));
    }
    static public Formula Get_Phi(double y_interface){
        return new Formula("BoundaryValues.Phi", AdditionalPrefixCode:GetPrefixCode(y_interface));
    }

};

## Create base control file
In this ControlFile basic configuration is defined.

In [ ]:
static XNSEC_Control XDG_pseudo2dCombustion(int DGp, int nCells = 6, string dbpath = null, bool MF = false) {
    XNSEC_Control C;
    if (MF) { // If it is a mixture fraction calculation, use the adequate control file
        C = new XNSEC_MF_Control();
        C.physicsMode = PhysicsMode.MixtureFraction;
    } else {
        C = new XNSEC_Control();
        C.physicsMode = PhysicsMode.Combustion;
    }
    // ==============
    // Solver configuration
    // ==============

    C.TimesteppingMode = AppControl._TimesteppingMode.Steady;
    C.NonLinearSolver.SolverCode = NonLinearSolverCode.Newton;
    C.NonLinearSolver.verbose = true;
    C.savetodb = dbpath == null ? false : true;
    C.DbPath = dbpath;


    C.PhysicalParameters.IncludeConvection = true;
    C.ThermalParameters.IncludeConvection = true;

    // Miscellaneous phyisical switches
    C.IncludeRecoilPressure = false;
    C.ChemicalReactionActive = false;
    C.NumberOfChemicalSpecies = C.EnableMassFractions ? 2 : 1;
    C.rhoOne = true; // If true, constant density will be considered in each phase (mainly for debugging)
    
    C.AdaptiveMeshRefinement = false;

    C.activeAMRlevelIndicators.Add(new BoSSS.Solution.LevelSetTools.SolverWithLevelSetUpdater.AMRonNarrowband() { maxRefinementLevel = 4, levelSet = 0 });
    C.SkipSolveAndEvaluateResidual = false;
    C.AgglomerationThreshold = 0.1;
    C.SetDGdegree(DGp);
    C.GravityDirection = new double[] { 0.0, 0.0, 0.0 };

    C.ImmediatePlotPeriod = 1;
    // ==============
    // Physical Parameters
    // ==============
    C.Reynolds = 1.0;
    C.Prandtl = 1.0;
    C.Schmidt = 1.0;
    C.Damk = 1e8 * 0 + 1e4;
    C.ReactionRateConstants = new double[] { C.Damk, 15, 1, 1 };
    C.HeatRelease = 4.0;

    C.smoothingFactor = 10 * 0;
    double y_interface = 7.14;
    double[] FuelInletMassFractions = new double[] { 1.0, 0.0, 0.0, 0.0, 0.0 };
    double[] OxidizerInletMassFractions = new double[] { 0.0, 1.0, 0.0, 0.0, 0.0 };
    C.StoichiometricCoefficients = new double[] { -1, -1, 1, 1, 0 };
    C.YFuelInlet = FuelInletMassFractions[0];
    C.YOxInlet = OxidizerInletMassFractions[1];
    C.s = 1;
    C.phi = C.s * C.YFuelInlet / C.YOxInlet;
    C.zSt = 1.0 / (1.0 + C.phi);
    double prescribedMass = 1e-2;

    C.prescribedMassflux = BoundaryValueFactory.Get_ConstantValue(prescribedMass);
    Console.WriteLine("The flamesheet is located at points with Z = " + C.zSt);

    C.PlotNewtonIterations = false;
    C.ThermalParameters.T_sat = 1.0; // boundary temperature

    C.NonLinearSolver.MaxSolverIterations = 10;
    C.NonLinearSolver.ConvergenceCriterion = 1e-7;
    C.ThermalParameters.hVap = 1;
    C.PhysicalParameters.rho_A = 2.0;
    C.PhysicalParameters.rho_B = 1;
    C.PhysicalParameters.mu_A = 1;
    C.PhysicalParameters.mu_B = 1;
    C.ThermalParameters.rho_A = 2.0;
    C.ThermalParameters.rho_B = 1;
    // Grid declaration
    // ===============
    C.SetGrid(GridFactory.GenerateGrid1D(nCells));

    bool m_BotPressureOutlet = true;
    bool m_TopPressureOutlet = false;

    // initial values and exact solution
    // =================================

    foreach (var spc in new[] { "A", "B" }) {
        C.AddInitialValue(VariableNames.VelocityX + "#" + spc, BoundaryValueFactory.Get_ConstantValue(0.0));
        C.AddInitialValue(VariableNames.VelocityY + "#" + spc, BoundaryValueFactory.Get_ConstantValue(0.2));
        C.AddInitialValue(VariableNames.Pressure + "#" + spc, BoundaryValueFactory.Get_ConstantValue(0.0));
        C.AddInitialValue(VariableNames.MixtureFraction + "#" + spc, BoundaryValueFactory.Get_ConstantValue(1.0));
        C.AddInitialValue(VariableNames.Temperature + "#" + spc, BoundaryValueFactory.Get_ConstantValue(1.0));
        C.AddInitialValue(VariableNames.MassFraction0 + "#" + spc, BoundaryValueFactory.Get_ConstantValue(1.0));
        C.AddInitialValue(VariableNames.MassFraction1 + "#" + spc, BoundaryValueFactory.Get_ConstantValue(0.0));
    }

    // C.Phi = GetPhi();
    C.AddInitialValue(VariableNames.LevelSetCG, BoundaryValueFactory.Get_Phi(y_interface));

    // boundary conditions
    // ===================

    if (!m_BotPressureOutlet) {
        C.AddBoundaryValue("velocity_inlet_bot", VariableNames.Velocity_d(0) + "#A", BoundaryValueFactory.Get_ConstantValue(0.0));
        C.AddBoundaryValue("velocity_inlet_bot", VariableNames.Velocity_d(1) + "#A", BoundaryValueFactory.Get_ConstantValue(prescribedMass / C.PhysicalParameters.rho_A));
        C.AddBoundaryValue("velocity_inlet_bot", VariableNames.MixtureFraction + "#A", BoundaryValueFactory.Get_ConstantValue(1.0));
        C.AddBoundaryValue("velocity_inlet_bot", VariableNames.Temperature + "#A", BoundaryValueFactory.Get_ConstantValue(1.0));
        C.AddBoundaryValue("velocity_inlet_bot", VariableNames.MassFraction0 + "#A", BoundaryValueFactory.Get_ConstantValue(1.0));
        C.AddBoundaryValue("velocity_inlet_bot", VariableNames.MassFraction1 + "#A", BoundaryValueFactory.Get_ConstantValue(0.0));
    } else {
        C.AddBoundaryValue("ScalarDirichlet_PressureOutlet_bot", VariableNames.Pressure + "#A", BoundaryValueFactory.Get_ConstantValue(m_RecoilPressure ? -prescribedMass * prescribedMass * (1 / C.PhysicalParameters.rho_A - 1 / C.PhysicalParameters.rho_B) : 0.0));
        C.AddBoundaryValue("ScalarDirichlet_PressureOutlet_bot", VariableNames.MixtureFraction + "#A", BoundaryValueFactory.Get_ConstantValue(1.0));
        C.AddBoundaryValue("ScalarDirichlet_PressureOutlet_bot", VariableNames.Temperature + "#A", BoundaryValueFactory.Get_ConstantValue(1.0));
        C.AddBoundaryValue("ScalarDirichlet_PressureOutlet_bot", VariableNames.MassFraction0 + "#A", BoundaryValueFactory.Get_ConstantValue(1.0));
        C.AddBoundaryValue("ScalarDirichlet_PressureOutlet_bot", VariableNames.MassFraction1 + "#A", BoundaryValueFactory.Get_ConstantValue(0.0));
    }
    if (!m_TopPressureOutlet) {
        C.AddBoundaryValue("velocity_inlet_top", VariableNames.Velocity_d(0) + "#A", BoundaryValueFactory.Get_ConstantValue(0.0));
        C.AddBoundaryValue("velocity_inlet_top", VariableNames.Velocity_d(1) + "#A", BoundaryValueFactory.Get_ConstantValue(prescribedMass / C.PhysicalParameters.rho_B * 1));
        C.AddBoundaryValue("velocity_inlet_top", VariableNames.MixtureFraction + "#A", BoundaryValueFactory.Get_ConstantValue(0.0));
        C.AddBoundaryValue("velocity_inlet_top", VariableNames.Temperature + "#A", BoundaryValueFactory.Get_ConstantValue(1.0));
        C.AddBoundaryValue("velocity_inlet_top", VariableNames.MassFraction0 + "#A", BoundaryValueFactory.Get_ConstantValue(0.0));
        C.AddBoundaryValue("velocity_inlet_top", VariableNames.MassFraction1 + "#A", BoundaryValueFactory.Get_ConstantValue(1.0));
    } else {
        C.AddBoundaryValue("ScalarDirichlet_PressureOutlet_top", VariableNames.Pressure + "#A", BoundaryValueFactory.Get_ConstantValue(0.0));
        C.AddBoundaryValue("ScalarDirichlet_PressureOutlet_top", VariableNames.MixtureFraction + "#A", BoundaryValueFactory.Get_ConstantValue(0.0));
        C.AddBoundaryValue("ScalarDirichlet_PressureOutlet_top", VariableNames.Temperature + "#A", BoundaryValueFactory.Get_ConstantValue(1.0));
        C.AddBoundaryValue("ScalarDirichlet_PressureOutlet_top", VariableNames.MassFraction0 + "#A", BoundaryValueFactory.Get_ConstantValue(0.0));
        C.AddBoundaryValue("ScalarDirichlet_PressureOutlet_top", VariableNames.MassFraction1 + "#A", BoundaryValueFactory.Get_ConstantValue(1.0));
    }
    return C;
}

## Starting the MixtureFraction simulation
Configuration  for the simulation using the mixture fraction approach, where an infinite reaction rate is assumed. Used to find adequate starting solution for the full problem.

In [ ]:
static public XNSEC_Control FS_XDG_pseudo2dCombustion(int DGp = 2, int nCells = 4,  string dbPath =null) {
    var C = XDG_pseudo2dCombustion(DGp, nCells, dbPath, true);
    C.physicsMode = PhysicsMode.MixtureFraction;
    C.ProjectName = "Pseudo1DCombustion";
    string name = C.ProjectName + "P" + DGp + "K" + nCells ;
    C.SessionName = "FS_" + name;

    C.UseSelfMadeTemporalOperator = false;
    //C.SkipSolveAndEvaluateResidual = true;

    Dictionary<string, Tuple<double, double>> Bounds = new Dictionary<string, Tuple<double, double>>();
    double eps = 0.05;
    Bounds.Add(VariableNames.MixtureFraction, new Tuple<double, double>(0 - eps, 1 + eps));
    C.VariableBounds = Bounds;
    //C.NonLinearSolver.MaxSolverIterations = 500;
    C.activeAMRlevelIndicators.Add(new AMR_onFlameSheet(C.zSt, 4));
    //C.AMR_startUpSweeps = 4;
    C.TimesteppingMode = BoSSS.Solution.Control.AppControl._TimesteppingMode.Steady;
    C.NoOfTimesteps = 2;
    C.savetodb = true;
    return C;
}

## Send and run jobs

In [ ]:

  Type solver_MF = typeof(BoSSS.Application.XNSEC.XNSEC_MixtureFraction);
  XNSEC_MF_Control C_MixtureFraction = (XNSEC_MF_Control)FS_XDG_pseudo2dCombustion(dgMF, nCells,myDb.Path);
  string jobName       = C_MixtureFraction.SessionName +"__2_0";
  Console.WriteLine(jobName);
  var oneJob           = new Job(jobName, solver_MF);
  oneJob.NumberOfMPIProcs =  1;
  oneJob.SetControlObject(C_MixtureFraction);
     oneJob.Activate(myBatch); 
    //oneJob.Activate(); 




The flamesheet is located at points with Z = 0.5
Grid Edge Tags changed.
FS_Pseudo1DCombustionP2K5__2_0
Deploying job FS_Pseudo1DCombustionP2K5__2_0 ... 
Deploying executables and additional files ...
Deployment directory: \\hpccluster\hpccluster-scratch\gutierrez\binaries\XDG_Droplet_Combustionn-XNSEC2022May31_134837
copied 56 files.
   written file: control.obj
   copied 'win\amd64' runtime.
deployment finished.



In [ ]:
BoSSSshell.WorkflowMgm.BlockUntilAllJobsTerminate();

All jobs finished.


## Starting the finite-rate chemistry simulation

Now that the simulation for an "infinite" reaction rate is done, we use it for initializing the system with finite reaction rate. The goal is to obtain solutions of the counter difussion flame for increasing strain values. We start with a low strain (bigger Dahmkoehler number), which is increased until extintion is (hopefully) found

In [ ]:
static public XNSEC_Control Full_XDG_pseudo2dCombustion(int DGp = 2, int nCells = 5, string dbPath = null) {
  

    var C = XDG_pseudo2dCombustion(DGp, nCells, dbPath, false);
    C.ProjectName = "Pseudo1DCombustion";
    string jobName = C.ProjectName + "P" + DGp + "K" + nCells;
    C.SessionName = "Full_" + jobName;

    C.physicsMode = PhysicsMode.Combustion;
    C.UseSelfMadeTemporalOperator = false;

    // limiting of variable values
    Dictionary<string, Tuple<double, double>> Bounds = new Dictionary<string, Tuple<double, double>>();
    double eps = 0.05;
    Bounds.Add(VariableNames.Temperature, new Tuple<double, double>(1.0 - eps, 10)); // Min temp should be the inlet temperature.
    Bounds.Add(VariableNames.MassFraction0, new Tuple<double, double>(0.0 - eps, 1.0 + eps)); // Between 0 and 1 per definition
    Bounds.Add(VariableNames.MassFraction1, new Tuple<double, double>(0.0 - eps, 1.0 + eps));
    Bounds.Add(VariableNames.MassFraction2, new Tuple<double, double>(0.0 - eps, 1.0 + eps));
    Bounds.Add(VariableNames.MassFraction3, new Tuple<double, double>(0.0 - eps, 1.0 + eps));
    C.VariableBounds = Bounds;

    C.NoOfTimesteps = 1; // The steady solution will be calculated again and do AMR
    C.myThermalWallType = SIPDiffusionTemperature.ThermalWallType.Adiabatic;

    //C.UseMixtureFractionsForCombustionInitialization = true;

    // Select the database
    DatabaseInfo dbi = DatabaseInfo.Open(C.DbPath);

    string RestartSessionName = ("FS_" + jobName);
    var sess = dbi.Sessions.Where(s => s.Name.Equals(RestartSessionName)).ToList();   //find the session where the restart should be done from
    if (sess.Count == 0) {
        Console.WriteLine("===========================");
        Console.WriteLine("No session found for restart. The pure mixing case will be calculated.");
        Console.WriteLine("===========================");
        C.ChemicalReactionActive = false;// true;
    } else {
        C.SetRestart(sess[0]);
        Console.WriteLine("===========================");
        Console.WriteLine("A session was found for restart in the database. The case with combustion will be calculated");
        if (sess.Count > 1) {
            Console.WriteLine("Warning: multiple jobs with the same name defined for restart. Using the most recent one");
        }
        C.ChemicalReactionActive = true;// true;
        Console.WriteLine("===========================");
    }
    return C;
}

In [ ]:
Type solver = typeof(BoSSS.Application.XNSEC.XNSEC);

Calculate the full solution for the initial value

In [ ]:

var C = Full_XDG_pseudo2dCombustion(dgMF, nCells,   myDb.Path );
string jobName = C.SessionName;
Console.WriteLine(jobName);
var oneJob = new Job(jobName, solver);
oneJob.NumberOfMPIProcs = 2;
oneJob.SetControlObject(C);
// oneJob.Activate(myBatch); 
oneJob.Activate(); 


The flamesheet is located at points with Z = 0.5
Grid Edge Tags changed.
An equivalent grid (2f611a61-8f62-4693-a1ef-b82d1058044c) is already present in the database -- the grid will not be saved.
A session was found for restart in the database. The case with combustion will be calculated
Full_Pseudo1DCombustionP2K5
Deploying job Full_Pseudo1DCombustionP2K5 ... 
Deploying executables and additional files ...
Deployment directory: \\hpccluster\hpccluster-scratch\gutierrez\binaries\XDG_Droplet_Combustionn-XNSEC2022May31_135031
copied 56 files.
   written file: control.obj
   copied 'win\amd64' runtime.
deployment finished.

